# Create exemplar blue/green/black patch files

Problem: first round of prediction said blue or green for many black files.  
Explanation: we were planning to filter black patches by color, 
but apparently we're no good at it. The CNN might do better.  
Solution: create a three-way classifier.  

Like DF_Filter.011, create subdirectories of train and valid for each class.  
Like DF_Filter.018, use slightly condensed code.  

## Results
Train  
* 14984 black  
* 2729 blue  
* 2358 green  

Valid  
* 3746 black
* 683 blue
* 590 green

In [1]:
import time
import os
import glob
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_TRAIN = "/home/jrm/Martinez/images/cache/train/"
DIR_IMAGES_VALID = "/home/jrm/Martinez/images/cache/valid/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG
IMAGE_SIZE = (PATCH_SIZE,PATCH_SIZE)
DF_Ypos = ['B7.','B15','D1.','D5.','E7.','E9.','F9.','G3.','H13','I1.','I5.','I13']
DF_Yneg = ['A3.','A5.','B13','C1.','C11','D3.','E5.','F3.','F7','F11','F13','F15','G15','H1.','H3.','H7.','H15']

In [3]:
def get_image_names(path,pattern):
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)

In [4]:
class patch_maker:
    def __init__(self):
        self.path=""
        self.w=0
        self.h=0
        self.im_width = 0
        self.im_height = 0
        self.patch_size = 10 # scalar, assumed square for now
        self.pixel_array = None
        self.output_dir = ""
        self.output_format='.jpg'
    def set_input_path(self,path):
        self.path=path
    def set_output_path(self,path):
        self.output_dir = path
    def set_patch_size(self,scalar):
        self.patch_size = scalar
    def load_pixel_array(self,filename,verbose=False):
        self.w=0
        self.h=0
        im = Image.open(self.path+filename)
        ima = np.array(im)   # convert to numpy
        self.im_width = ima.shape[0]
        self.im_height = ima.shape[1]    
        if verbose:
            print(filename, ima.size, ima.shape)
        self.pixel_array = ima
        return ima
    def get_next_patch(self): 
        IM_WIDTH = self.im_width
        IM_HEIGHT = self.im_height
        PIXEL_ARRAY = self.pixel_array
        patch = None
        if self.w+self.patch_size > IM_WIDTH:
            self.h += self.patch_size
            self.w = 0
        if self.w+self.patch_size <= IM_WIDTH and self.h+self.patch_size <= IM_HEIGHT:
            patch = PIXEL_ARRAY[self.w:self.w+self.patch_size, self.h:self.h+self.patch_size]
            self.w += self.patch_size
        return patch
    def save_patch (self, prefix, pnum, patch):
        ext = self.output_format
        path= self.output_dir+prefix+'.'+str(pnum)+ext
        im =  Image.fromarray(patch)
        im.save(path)

In [5]:
MIN_GREEN_INTENSITY=np.array( [0,35,0] )
MIN_BLUE_INTENSITY =np.array( [0,0,30] )
MAX_BLACK_INTENSITY=np.array( [10,10,10] )
pm = patch_maker()
pm.set_input_path(DIR_IMAGES_RAW)
pm.set_patch_size(PATCH_SIZE)
pm.set_output_path(DIR_IMAGES_TRAIN)  # later, move 20% of files to VALID
for filename in FILENAMES_IMAGES_RAW:
    prefix = filename[0:3]
    blue = 0
    black = 0
    green = 0
    if prefix in DF_Yneg:
        if prefix[2] == '.':  # we have prefixes like F1. and F13
            prefix=prefix[0:2]+'_'   # avoid the extra period
        print(filename,end=" ")
        pixels = pm.load_pixel_array(filename)
        patch = pm.get_next_patch()
        while patch is not None:
            num_green_pixels = np.sum(np.all(patch >= MIN_GREEN_INTENSITY,axis=2))
            num_blue_pixels  = np.sum(np.all(patch >= MIN_BLUE_INTENSITY,axis=2))
            num_black_pixels = np.sum(np.all(patch <= MAX_BLACK_INTENSITY,axis=2))
            if num_black_pixels>=50000 and num_blue_pixels<=1000 and num_green_pixels<1000:
                black += 1
                pm.save_patch('black/'+prefix,black,patch)
            if num_blue_pixels>=10000 and num_blue_pixels>num_green_pixels*20:
                blue += 1
                pm.save_patch('blue/'+prefix,blue,patch)
            elif num_green_pixels>=15000 and num_green_pixels>num_blue_pixels*2:
                green += 1
                pm.save_patch('green/'+prefix,green,patch)
            patch = pm.get_next_patch()
        print(" Write %d black, %d blue, %d green patches."%(black,blue,green))
print('done')

F15.DF1.135.tif  Write 1078 black, 262 blue, 34 green patches.
H3.DF1.27.tif  Write 25 black, 30 blue, 110 green patches.
C11.DF1.96.tif  Write 2 black, 16 blue, 292 green patches.
C1.DF1.01.tif  Write 0 black, 293 blue, 69 green patches.
F11.DF1.94.tif  Write 1127 black, 375 blue, 54 green patches.
H7.DF1.62.tif  Write 1166 black, 228 blue, 178 green patches.
A3.DF1.26.tif  Write 328 black, 1482 blue, 1 green patches.
A5.DF1.45.tif  Write 0 black, 6 blue, 454 green patches.
H15.DF1.132.tif  Write 0 black, 0 blue, 66 green patches.
G15.DF1.133.tif  Write 0 black, 0 blue, 239 green patches.
F3.DF1.22.tif  Write 0 black, 41 blue, 430 green patches.
B13.DF1.115.tif  Write 187 black, 407 blue, 70 green patches.
D3.DF1.20.tif  Write 0 black, 4 blue, 325 green patches.
E5.DF1.42.tif  Write 17 black, 266 blue, 31 green patches.
H1.DF1.07.tif  Write 0 black, 0 blue, 593 green patches.
done


In [6]:
from random import sample
for color in ('blue','green','black'):
    fromdir = DIR_IMAGES_TRAIN+"/"+color+"/"
    todir   = DIR_IMAGES_VALID+"/"+color+"/"
    files = os.listdir(fromdir)
    total = len(files)
    selected = sample(files, int(0.20*total))
    for vfile in selected:
        os.rename(fromdir+vfile,todir+vfile)
print("Done")

Done
